In [1]:
import itertools
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import quad

In [2]:
def generate_file_path(choices):
    """
    Generates a file path based on commnon choices and variable sample, years
    """
    #  template with placeholders
    template = "/eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/{Sample}/{Data_or_Simulation}{Year}{Track_type}{Magnet_polarity}/{Sample}_procTuple.root"
    
    # Replace placeholders with actual values
    file_path = template
    for key, value in choices.items():
        file_path = file_path.replace("{" + key + "}", value)

    return file_path

# Define common choices
common_choices = {
    'Data_or_Simulation': 'MC',
    'Track_type': 'LL',
    'Magnet_polarity': 'MU'
}

# Define samples and their specific year requirements if any
samples = [
    ("Lb2LMM", '18'),
    ("Lb2LJPsMM", '18'), 
    ("Lb2L1520MM", '18'),
    ("Lb2L1520JPsMM", '18'),
    ("Xib02Xi0MM", '18'),
    ("Bd2KSJPsMM", '18'),
    ("Bd2KSMM", '18'),
    ("Xib2XiMM", '18')
]
#     ("Omb2OmMM", '16')

# Generate file paths for each sample
file_paths = {}
for sample, year in samples:
    # Create choices for the sample
    choices = {**common_choices, 'Sample': sample, 'Year': year}
    
    # Generate the file path for the  sample
    file_paths[f'file_{sample}'] = generate_file_path(choices)

# Print the generated file paths
for file_name, file_path in file_paths.items():
    print(f"{file_name}: {file_path}")

# This opens the files with uproot
file_handles = {}
for file_name, file_path in file_paths.items():
    try:
        file_handles[file_name] = uproot.open(file_path)
        print(f"Opened {file_name}")
    except Exception as e:
        print(f"Failed to open {file_name}: {e}")

# Now we can access the files with the file handles
# Example:
# for file_name, file_handle in file_handles.items():
#     print(f"Opened {file_name}")

file_Lb2LMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Lb2LMM/MC18LLMU/Lb2LMM_procTuple.root
file_Lb2LJPsMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Lb2LJPsMM/MC18LLMU/Lb2LJPsMM_procTuple.root
file_Lb2L1520MM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Lb2L1520MM/MC18LLMU/Lb2L1520MM_procTuple.root
file_Lb2L1520JPsMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Lb2L1520JPsMM/MC18LLMU/Lb2L1520JPsMM_procTuple.root
file_Xib02Xi0MM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Xib02Xi0MM/MC18LLMU/Xib02Xi0MM_procTuple.root
file_Bd2KSJPsMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Bd2KSJPsMM/MC18LLMU/Bd2KSJPsMM_procTuple.root
file_Bd2KSMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Bd2KSMM/MC18LLMU/Bd2KSMM_procTuple.root
file_Xib2XiMM: /eos/lhcb/wg/RD/Lb2Lll/RLambda/Tuples/v206/TupleProcess_MM_MVA/Xib2XiMM/MC18LLMU/Xib2XiMM_procTuple.root
Opened file_L

In [3]:
# Iterate through the created file handles and print keys (to see what is in here)
for file_name, file_handle in file_handles.items():
    print(f"{file_name}.keys = {file_handle.keys()}")
    
# Now we know that all files have the same two keys 

file_Lb2LMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;3']
file_Lb2LJPsMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;2']
file_Lb2L1520MM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']
file_Lb2L1520JPsMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']
file_Xib02Xi0MM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']
file_Bd2KSJPsMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']
file_Bd2KSMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']
file_Xib2XiMM.keys = ['Lb2JpsiL_mmTuple;1', 'Lb2JpsiL_mmTuple/DecayTree;1']


In [4]:
# Create a dictionary to store branches all files
branches = {}

for file_name, file_handle in file_handles.items():
    # get the branch from the file handle
    branch = file_handle['Lb2JpsiL_mmTuple/DecayTree'].arrays()
    
    # naming the branch 
    branch_name = f"{file_name.split('_')[1]}"  # Extracting the sample name as all files start with ''file
    
    # Store the branch in the dictionary
    branches[branch_name] = branch

# Print the branches for each file to check
for branch_name, branch in branches.items():
    print(f"{branch_name} = {branch}")

Lb2LMM = [{Lb_ENDVERTEX_X: 0.821, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.892}]
Lb2LJPsMM = [{Lb_ENDVERTEX_X: 0.863, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.69}]
Lb2L1520MM = [{Lb_ENDVERTEX_X: 1.04, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.931}]
Lb2L1520JPsMM = [{Lb_ENDVERTEX_X: 0.629, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.109}]
Xib02Xi0MM = [{Lb_ENDVERTEX_X: 0.75, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.979}]
Bd2KSJPsMM = [{Lb_ENDVERTEX_X: 0.707, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.968}]
Bd2KSMM = [{Lb_ENDVERTEX_X: 0.585, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.95}]
Xib2XiMM = [{Lb_ENDVERTEX_X: -0.459, ... xgboost_MM_TriggerL0L_YearR2p2_TrackLL: 0.658}]


In [5]:
# The transverse momentum does not exist in there so must make it ourselves for proton and pion 

def calculate_protonpt(sample):
    prx_var = 'Lb_DTF_L0_JPs_PV_Lambda0_pplus_PX_0'
    pry_var = 'Lb_DTF_L0_JPs_PV_Lambda0_pplus_PY_0'
    prt_var = 'Lb_DTF_L0_JPs_PV_Lambda0_pplus_PT_0'
    
    prx_var1 = 'Lb_DTF_L0_PV_Lambda0_pplus_PX_0'
    pry_var1 = 'Lb_DTF_L0_PV_Lambda0_pplus_PY_0'
    prt_var1 = 'Lb_DTF_L0_PV_Lambda0_pplus_PT_0'
    
    prx_values = branches[sample][prx_var]
    pry_values = branches[sample][pry_var]
    prt_values = np.sqrt(prx_values**2 + pry_values**2)
    branches[sample][prt_var] = prt_values
    
    prx_values1 = branches[sample][prx_var1]
    pry_values1 = branches[sample][pry_var1]
    prt_values1 = np.sqrt(prx_values1**2 + pry_values1**2)
    branches[sample][prt_var1] = prt_values1
    
# Calculate new proton pT variables for each sample
for branch_name in branches:
    calculate_protonpt(branch_name)

# for the pions
def calculate_pionpt(sample):
    pix_var = 'Lb_DTF_L0_JPs_PV_Lambda0_piplus_PX_0'
    piy_var = 'Lb_DTF_L0_JPs_PV_Lambda0_piplus_PY_0'
    pit_var = 'Lb_DTF_L0_JPs_PV_Lambda0_piplus_PT_0'

    pix_var1 = 'Lb_DTF_L0_PV_Lambda0_piplus_PX_0'
    piy_var1 = 'Lb_DTF_L0_PV_Lambda0_piplus_PY_0'
    pit_var1 = 'Lb_DTF_L0_PV_Lambda0_piplus_PT_0'
    
    pix_values = branches[sample][pix_var]
    piy_values = branches[sample][piy_var]
    pit_values = np.sqrt(pix_values**2 + piy_values**2)
    branches[sample][pit_var] = pit_values
    
    pix_values1 = branches[sample][pix_var1]
    piy_values1 = branches[sample][piy_var1]
    pit_values1 = np.sqrt(pix_values1**2 + piy_values1**2)
    branches[sample][pit_var1] = pit_values1
    
# Calculate new pion pT variables for each sample
for branch_name in branches:
    calculate_pionpt(branch_name)

/cvmfs/sft.cern.ch/lcg/views/LCG_105a_swan/x86_64-el9-gcc13-opt/lib/python3.9/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: overflow encountered in power
  result = getattr(ufunc, method)(


In [ ]:
# Now for the weights: 
for sample in samples_to_plot:
    branches[sample]['combined_weight'] = (
        branches[sample]['w_GBR_kinonly_LL'] *  # Use w_GBR_kinonly_DD if DD sample
        branches[sample]['w_tracking_P_common_Default'] *
        branches[sample]['w_tracking_Pi_common_Default'] *
        branches[sample]['w_tracking_L1_common_Default'] *
        branches[sample]['w_tracking_L2_common_Default'] *
        branches[sample]['L1_wPIDEffCalib_Muon_nominal'] *
        branches[sample]['L2_wPIDEffCalib_Muon_nominal'] *
        branches[sample]['combined_wTrigL0_correction_nominal']
    )
tot_weight = branches[sample]['combined_weight']

In [ ]:
samples_to_plot = list(branches.keys())
mass_up = 1000 * np.sqrt(8)  # so that we plot below q2 = 8 GeV^2/c^4
mass_muon = 105.66  # in MeV/c^2


lqcounter_before= {}
lqcounter_before_error = {}
lqcounter_after = {}
lqcounter_after_error = {}

hqcounter_before = {}
hqcounter_before_error = {}
hqcounter_afterw = {}
hqcounter_after_error = {}

scale_low = {}
scale_high = {}

# Calculate histograms and errors, and plot them
for sample in samples_to_plot:
    tot_weight = branches[sample]['combined_weight']
    
    # Set the q2 upper and lower values for low q2
    lqmass_hcut = branches[sample]['JPs_MM'] < mass_up
    lqmass_lcut = 2 * mass_muon < branches[sample]['JPs_MM']
    lq2intmask = lqmass_hcut & lqmass_lcut
    
    # Set the q2 upper and lower values for high q2
    hqmass_hcut = branches[sample]['JPs_MM'] > mass_up
    hqmass_lcut = 1000*np.sqrt(11) > branches[sample]['JPs_MM']
    hq2intmask = hqmass_hcut & hqmass_lcut
    
    
    x1 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_ID_0'] == -211.0
    x2 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_ID_0'] == 2212.0
    x3 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_PT_0'] > 500
    x4 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_PT_0'] >500
    x5 = branches[sample]['JPs_PT'] > 1000
    x6 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] < 1115+20
    x7 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] > 1115-20
    x8 = branches[sample]['Lb_DTF_Lb_L0_PV_ctau_0'] < 100
    x9 = branches[sample]['Lb_DTF_Lb_PV_chi2_0'] < 30
    ######################### unique to normalisation mode
    x10 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_piplus_ID_0'] == -211.0
    x20 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_pplus_ID_0'] == 2212.0
    x30 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_pplus_PT_0'] > 500
    x40 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_piplus_PT_0'] >500
    x60 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_M_0'] < 1115+20
    x70 = branches[sample]['Lb_DTF_L0_JPs_PV_Lambda0_M_0'] > 1115-20
    
    total_cut1 = lq2intmask & x1 & x2 & x3 & x4 & x5 & x6 & x7 & x8 & x9
    total_cut2 = hq2intmask & x10 & x20 & x30 & x40 & x5 & x60 & x70 & x8 & x9
    
    # Calculate histogram before and after cutting low q2
    hist10, bin_edges10 = np.histogram(branches[sample]['JPs_MM'][lq2intmask], bins=50, weights=tot_weight[lq2intmask])
    weights_squared10 = tot_weight[lq2intmask] ** 2
    hist_squared10, _ = np.histogram(branches[sample]['JPs_MM'][lq2intmask], bins=50, weights=weights_squared10)
    errors10 = np.sqrt(hist_squared10)
    bin_centers10 = (bin_edges10[:-1] + bin_edges10[1:]) / 2
    bin_width10 = bin_centers10[1]-bin_centers10[0]
    
    hist1, bin_edges1 = np.histogram(branches[sample]['JPs_MM'][total_cut1], bins=50, weights=tot_weight[total_cut1])
    weights_squared1 = tot_weight[total_cut1] ** 2
    hist_squared1, _ = np.histogram(branches[sample]['JPs_MM'][total_cut1], bins=50, weights=weights_squared1)
    errors1 = np.sqrt(hist_squared1)
    bin_centers1 = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    bin_width1 = bin_centers1[1]-bin_centers1[0]
    
    # Calculate histogram for high before and after cutting q2
    hist20, bin_edges20 = np.histogram(branches[sample]['JPs_MM'][hq2intmask], bins=50, weights=tot_weight[hq2intmask])
    weights_squared20 = tot_weight[hq2intmask] ** 2
    hist_squared20, _ = np.histogram(branches[sample]['JPs_MM'][hq2intmask], bins=50, weights=weights_squared20)
    errors20 = np.sqrt(hist_squared20)
    bin_centers20 = (bin_edges20[:-1] + bin_edges20[1:]) / 2
    bin_width20 = bin_centers20[1]-bin_centers20[0]
    
    hist2, bin_edges2 = np.histogram(branches[sample]['JPs_MM'][total_cut2], bins=50, weights=tot_weight[total_cut2])
    weights_squared2 = tot_weight[total_cut2] ** 2
    hist_squared2, _ = np.histogram(branches[sample]['JPs_MM'][total_cut2], bins=50, weights=weights_squared2)
    errors2 = np.sqrt(hist_squared2)
    bin_centers2 = (bin_edges2[:-1] + bin_edges2[1:]) / 2
    bin_width2 = bin_centers2[1]-bin_centers2[0]
    
    ############################
    # Count the number of events and errors before and after cutting for both modes: 
    
    lqcounter_before[sample] = np.sum(hist10)
    lqcounter_before_error[sample] = np.sqrt(np.sum(errors10**2))
    lqcounter_after[sample] = np.sum(hist1)
    lqcounter_after_error[sample] = np.sqrt(np.sum(errors1**2))

    hqcounter_before[sample] = np.sum(hist20)
    hqcounter_before_error[sample] = np.sqrt(np.sum(errors20**2))
    hqcounter_after[sample] = np.sum(hist2)
    hqcounter_after_error[sample] = np.sqrt(np.sum(errors2**2))
    
    # Now 'normalise everything' For this give everything the same number of events before cutting: 
    # notice that in the low q2 region every histogram contains 5000 events and for high q2 every one 
    # contains 15000 events. Now that all the samples are put on equal footing the number of events (of the 5000/15000)
    # that pass all selections can be obtained by multiplying all the counters by the scales:
    scale_low[sample] = 5000/ lqcounter_before[sample]
    scale_high[sample] = 15000/ hqcounter_before[sample]

print(hqcounter_before["Lb2LJPsMM"]*scale_high["Lb2LJPsMM"])
print("Low q2")
for sample in samples_to_plot:
    print(f"{sample}: number of events before: {scale_low[sample]*lqcounter_before[sample]}, error : {scale_low[sample]*lqcounter_before_error[sample]} ")
    print(f"{sample}: number of events after: {scale_low[sample]*lqcounter_after[sample]}, error : {scale_low[sample]*lqcounter_after_error[sample]} ")
print("High q2")
for sample in samples_to_plot:
    print(f"{sample}: number of events before: {scale_high[sample]*hqcounter_before[sample]}, error : {scale_high[sample]*hqcounter_before_error[sample]} ")
    print(f"{sample}: number of events after: {scale_high[sample]*hqcounter_after[sample]}, error : {scale_high[sample]*hqcounter_after_error[sample]} ")
      

In [ ]:
# the following code is quite a lot of repition but it wuoldn't work well otherwise

samples_to_plot = list(branches.keys())
mass_up = 1000 * np.sqrt(8)  # so that we plot below q2 = 8 GeV^2/c^4
mass_muon = 105.66  # in MeV/c^2

sample_legend = {
    "Lb2LMM": r"$\Lambda_b \rightarrow \Lambda \; \mu \mu$",
    "Lb2LJPsMM": r"$\Lambda_b \rightarrow \Lambda \; J/\psi \, (\rightarrow \mu \mu )$",
    "Lb2L1520MM": r"$\Lambda_b \rightarrow \Lambda(1520) \mu \mu$",
    "Lb2L1520JPsMM": r"$\Lambda_b \rightarrow \Lambda(1520) \; J/\psi \, (\rightarrow \mu \mu )$",
    "Xib02Xi0MM": r"$\Xi_b^0 \rightarrow \Xi^0 \; \mu \mu$",
    "Bd2KSJPsMM": r"$B_d \rightarrow K_S \; J/\psi \, (\rightarrow \mu \mu )$",
    "Bd2KSMM": r"$B_d \rightarrow K_S \; \mu \mu$",
    "Xib2XiMM": r"$\Xi_b \rightarrow \Xi \; \mu \mu$"
}

samples = [
    ("Lb2LMM", '18'),
    ("Lb2LJPsMM", '18'), 
    ("Lb2L1520MM", '18'),
    ("Lb2L1520JPsMM", '18'),
    ("Xib02Xi0MM", '18'),
    ("Bd2KSJPsMM", '18'),
    ("Bd2KSMM", '18'),
    ("Xib2XiMM", '18')
]

plt.figure(figsize = (12,9))
for sample in samples_to_plot:
    tot_weight = branches[sample]['combined_weight']
    
    # Set the q2 upper and lower values for low q2
    lqmass_hcut = branches[sample]['JPs_MM'] < mass_up
    lqmass_lcut = 2 * mass_muon < branches[sample]['JPs_MM']
    lq2intmask = lqmass_hcut & lqmass_lcut
    
    # Set the q2 upper and lower values for high q2
    hqmass_hcut = branches[sample]['JPs_MM'] > mass_up
    hqmass_lcut = 1000*np.sqrt(11) > branches[sample]['JPs_MM']
    hq2intmask = hqmass_hcut & hqmass_lcut

    x1 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_ID_0'] == -211.0
    x2 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_ID_0'] == 2212.0
    x3 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_PT_0'] > 500
    x4 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_PT_0'] >500
    x5 = branches[sample]['JPs_PT'] > 1000
    x6 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] < 1115+20
    x7 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] > 1115-20
    x8 = branches[sample]['Lb_DTF_Lb_L0_PV_ctau_0'] < 100
    x9 = branches[sample]['Lb_DTF_Lb_PV_chi2_0'] < 30
    
    total_cut1 = lq2intmask & x1 & x2 & x3 & x4 & x5 & x6 & x7 & x8 & x9
    
    hist1, bin_edges1 = np.histogram(branches[sample]['JPs_MM'][total_cut1], bins=50, weights=tot_weight[total_cut1])
    weights_squared1 = tot_weight[total_cut1] ** 2
    hist_squared1, _ = np.histogram(branches[sample]['JPs_MM'][total_cut1], bins=50, weights=weights_squared1)
    errors1 = np.sqrt(hist_squared1)
    bin_centers1 = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    bin_width1 = bin_centers1[1]-bin_centers1[0]
    
    if np.sum(total_cut1) > 0:  # at some points not doing this caused problem
        plt.hist(branches[sample]['JPs_MM'][total_cut1], bins=50, weights=tot_weight[total_cut1], histtype='step', label = sample_legend[sample])
        plt.xlabel('Dimuon Invariant Mass $(MeV/c^2)$', fontsize=15)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel(f'Counts / {bin_width1:.1f} $MeV/c^2$', fontsize=15)
        plt.legend(bbox_to_anchor=(1, 1), fontsize=15)
        plt.title('Low $q^2$ range: weighted histogram of Invariant Mass for different simulation samples', fontsize=15)
    
        if sample == "Lb2LMM":
            plt.errorbar(bin_centers1, hist1, yerr=errors1, color='black', ls='none' , label = 'Dimuon mode error bars')
    
plt.show()

plt.figure(figsize = (12,9))
for sample in samples_to_plot:
    tot_weight = branches[sample]['combined_weight']
    
    # Set the q2 upper and lower values for low q2
    lqmass_hcut = branches[sample]['JPs_MM'] < mass_up
    lqmass_lcut = 2 * mass_muon < branches[sample]['JPs_MM']
    lq2intmask = lqmass_hcut & lqmass_lcut
    
    # Set the q2 upper and lower values for high q2
    hqmass_hcut = branches[sample]['JPs_MM'] > mass_up
    hqmass_lcut = 1000*np.sqrt(11) > branches[sample]['JPs_MM']
    hq2intmask = hqmass_hcut & hqmass_lcut

    x1 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_ID_0'] == -211.0
    x2 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_ID_0'] == 2212.0
    x3 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_PT_0'] > 500
    x4 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_PT_0'] >500
    x5 = branches[sample]['JPs_PT'] > 1000
    x6 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] < 1115+20
    x7 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] > 1115-20
    x8 = branches[sample]['Lb_DTF_Lb_L0_PV_ctau_0'] < 100
    x9 = branches[sample]['Lb_DTF_Lb_PV_chi2_0'] < 30
    
    total_cut1 = lq2intmask & x1 & x2 & x3 & x4 & x5 & x6 & x7 & x8 & x9
    
    # Calculate histogram before and after cutting low q2
    hist10, bin_edges10 = np.histogram(branches[sample]['JPs_MM'][lq2intmask], bins=50, weights=tot_weight[lq2intmask])
    weights_squared10 = tot_weight[lq2intmask] ** 2
    hist_squared10, _ = np.histogram(branches[sample]['JPs_MM'][lq2intmask], bins=50, weights=weights_squared10)
    errors10 = np.sqrt(hist_squared10)
    bin_centers10 = (bin_edges10[:-1] + bin_edges10[1:]) / 2
    bin_width10 = bin_centers10[1]-bin_centers10[0]
    
    if np.sum(lq2intmask) > 0:
        plt.hist(branches[sample]['JPs_MM'][lq2intmask], bins=50, weights=tot_weight[lq2intmask], histtype='step', label = sample_legend[sample])
        plt.xlabel('Dimuon Invariant Mass $(MeV/c^2)$', fontsize=15)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel(f'Counts / {bin_width10:.1f} $MeV/c^2$', fontsize=15)
        plt.legend(bbox_to_anchor=(1, 1), fontsize=15)
        plt.title('Low $q^2$ range: weighted histogram of Invariant Mass for different simulation samples', fontsize=15)
    
        if sample == "Lb2LMM":
            plt.errorbar(bin_centers10, hist10, yerr=errors10, color='black', ls='none', label = 'Dimuon mode error bars')
    
plt.show()

plt.figure(figsize = (12,9))
for sample in samples_to_plot:
    tot_weight = branches[sample]['combined_weight']
    
    # Set the q2 upper and lower values for low q2
    lqmass_hcut = branches[sample]['JPs_MM'] < mass_up
    lqmass_lcut = 2 * mass_muon < branches[sample]['JPs_MM']
    lq2intmask = lqmass_hcut & lqmass_lcut
    
    # Set the q2 upper and lower values for high q2
    hqmass_hcut = branches[sample]['JPs_MM'] > mass_up
    hqmass_lcut = 1000*np.sqrt(11) > branches[sample]['JPs_MM']
    hq2intmask = hqmass_hcut & hqmass_lcut

    x1 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_ID_0'] == -211.0
    x2 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_ID_0'] == 2212.0
    x3 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_PT_0'] > 500
    x4 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_PT_0'] >500
    x5 = branches[sample]['JPs_PT'] > 1000
    x6 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] < 1115+20
    x7 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] > 1115-20
    x8 = branches[sample]['Lb_DTF_Lb_L0_PV_ctau_0'] < 100
    x9 = branches[sample]['Lb_DTF_Lb_PV_chi2_0'] < 30
    
    total_cut2 = hq2intmask & x1 & x2 & x3 & x4 & x5 & x6 & x7 & x8 & x9
    
    
    hist2, bin_edges2 = np.histogram(branches[sample]['JPs_MM'][total_cut2], bins=50, weights=tot_weight[total_cut2])
    weights_squared2 = tot_weight[total_cut2] ** 2
    hist_squared2, _ = np.histogram(branches[sample]['JPs_MM'][total_cut2], bins=50, weights=weights_squared2)
    errors2 = np.sqrt(hist_squared2)
    bin_centers2 = (bin_edges2[:-1] + bin_edges2[1:]) / 2
    bin_width2 = bin_centers2[1]-bin_centers2[0]
    
    if np.sum(total_cut2) > 0:
        plt.hist(branches[sample]['JPs_MM'][total_cut2], bins=50, weights=tot_weight[total_cut2], histtype='step', label = sample_legend[sample])
        plt.xlabel('Dimuon Invariant Mass $(MeV/c^2)$', fontsize=15)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel(f'Counts / {bin_width2:.1f} $MeV/c^2$', fontsize=15)
        plt.legend(bbox_to_anchor=(1, 1), fontsize=13)
        plt.title('High $q^2$ range: weighted histogram of Invariant Mass for different simulation samples', fontsize=15)
    
        if sample == "Lb2LJPsMM":
            plt.errorbar(bin_centers2, hist2, yerr=errors2, color='black', ls='none', label = 'Normalisation mode error bars')
    
plt.show()
plt.figure(figsize = (12,9))
for sample in samples_to_plot:
    tot_weight = branches[sample]['combined_weight']
    
    # Set the q2 upper and lower values for low q2
    lqmass_hcut = branches[sample]['JPs_MM'] < mass_up
    lqmass_lcut = 2 * mass_muon < branches[sample]['JPs_MM']
    lq2intmask = lqmass_hcut & lqmass_lcut
    
    # Set the q2 upper and lower values for high q2
    hqmass_hcut = branches[sample]['JPs_MM'] > mass_up
    hqmass_lcut = 1000*np.sqrt(11) > branches[sample]['JPs_MM']
    hq2intmask = hqmass_hcut & hqmass_lcut
    
    x1 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_ID_0'] == -211.0
    x2 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_ID_0'] == 2212.0
    x3 = branches[sample]['Lb_DTF_L0_PV_Lambda0_pplus_PT_0'] > 500
    x4 = branches[sample]['Lb_DTF_L0_PV_Lambda0_piplus_PT_0'] >500
    x5 = branches[sample]['JPs_PT'] > 1000
    x6 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] < 1115+20
    x7 = branches[sample]['Lb_DTF_L0_PV_Lambda0_M_0'] > 1115-20
    x8 = branches[sample]['Lb_DTF_Lb_L0_PV_ctau_0'] < 100
    x9 = branches[sample]['Lb_DTF_Lb_PV_chi2_0'] < 30
    
    total_cut2 = hq2intmask & x1 & x2 & x3 & x4 & x5 & x6 & x7 & x8 & x9
    
    
    hist20, bin_edges20 = np.histogram(branches[sample]['JPs_MM'][hq2intmask], bins=50, weights=tot_weight[hq2intmask])
    weights_squared20 = tot_weight[hq2intmask] ** 2
    hist_squared20, _ = np.histogram(branches[sample]['JPs_MM'][hq2intmask], bins=50, weights=weights_squared20)
    errors20 = np.sqrt(hist_squared20)
    bin_centers20 = (bin_edges20[:-1] + bin_edges20[1:]) / 2
    bin_width20 = bin_centers20[1]-bin_centers20[0]
        
    if np.sum(hq2intmask) > 0:
        plt.hist(branches[sample]['JPs_MM'][hq2intmask], bins=50, weights=tot_weight[hq2intmask], histtype='step', label = sample_legend[sample])
        plt.xlabel('Dimuon Invariant Mass $(MeV/c^2)$', fontsize=15)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel(f'Counts  / {bin_width20:.1f} $MeV/c^2$', fontsize=15)
        plt.legend(bbox_to_anchor=(1, 1), fontsize=13)
        plt.title('High $q^2$ range: weighted histogram of Invariant Mass for different simulation samples', fontsize=15)
    
        if sample == "Lb2LJPsMM":
            plt.errorbar(bin_centers20, hist20, yerr=errors20, color='black', ls='none', label = 'Normalisation mode error bars')
    
plt.show()